In [8]:
# Version 2 + Bug fix - thanks to @chinhuic

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from torch.utils import data
import math

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
#print(os.listdir("../input/nvidiaapex/repository/NVIDIA-apex-39e153a"))
#print(os.listdir("../input/glove-global-vectors-for-word-representation"))
#print(os.listdir("../input/jigsaw-unintended-bias-in-toxicity-classification"))
#print(os.listdir("../input/fasttext-crawl-300d-2m"))

# Any results you write to the current directory are saved as output.

In [9]:
# Installing Nvidia Apex
#! pip install -v --global-option="--cpp_ext" --global-option="--cuda_ext" ../input/nvidiaapex/repository/NVIDIA-apex-39e153a

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:

device=torch.device('cuda')

In [12]:
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
EPOCHS = 1
#Data_dir="../input/jigsaw-unintended-bias-in-toxicity-classification"
Data_dir="../input/"
Input_dir = "../input"
WORK_DIR = "../working/"
num_to_load=1804874                        #Train size to match time limit
#num_to_load=1000
valid_size=0                           #Validation Size
TOXICITY_COLUMN = 'target'

In [13]:
# Add the Bart Pytorch repo to the PATH
# using files from: https://github.com/huggingface/pytorch-pretrained-BERT
package_dir_a = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.insert(0, package_dir_a)

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam


/home/curtis/miniconda3/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping
/home/curtis/miniconda3/lib/python3.7/site-packages/socks.py:58: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Callable
/home/curtis/miniconda3/lib/python3.7/site-packages/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message
/home/curtis/miniconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, 

In [16]:
# Translate model from tensorflow to pytorch
#BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-24_H-1024_A-16/' # cicero
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /home/curtis/Desktop/kaggle/jigsaw/input/bert-pretrained-models/uncased_L-24_H-1024_A-16/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [1024]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [1024]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 1024]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 1024]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 1024]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_0/attenti

Loading TF weight bert/encoder/layer_17/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_17/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_17/intermediate/dense/kernel with shape [1024, 4096]
Loading TF weight bert/encoder/layer_17/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_17/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_17/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_17/output/dense/kernel with shape [4096, 1024]
Loading TF weight bert/encoder/layer_18/attention/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_18/attention/output/LayerNorm/gamma with shape [1024]
Loading TF weight bert/encoder/layer_18/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_18/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_18/attention/self

Loading TF weight bert/encoder/layer_6/attention/output/dense/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/output/dense/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/attention/self/key/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/key/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/attention/self/query/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/query/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/attention/self/value/bias with shape [1024]
Loading TF weight bert/encoder/layer_6/attention/self/value/kernel with shape [1024, 1024]
Loading TF weight bert/encoder/layer_6/intermediate/dense/bias with shape [4096]
Loading TF weight bert/encoder/layer_6/intermediate/dense/kernel with shape [1024, 4096]
Loading TF weight bert/encoder/layer_6/output/LayerNorm/beta with shape [1024]
Loading TF weight bert/encoder/layer_6/output/LayerNorm

Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_21', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'self', 'key', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22', 'attention', 'self', 'key', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_22'

'../working/bert_config.json'

In [17]:
os.listdir("../working")

['pytorch_model.bin', 'bert_config.json']

In [19]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig

#bert_config = BertConfig('../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'+'bert_config.json')
bert_config = BertConfig('../input/bert-pretrained-models/uncased_L-24_H-1024_A-16/bert_config.json')


In [20]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            first = int(math.ceil(max_seq_length * 0.25)) #first 25%
            last = max_seq_length - first
            tokens_first = tokens_a[:first]
            tokens_last = tokens_a[-last:]
            tokens_a = tokens_first + tokens_last
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
       
        all_tokens.append(one_token)
    print("Number of sequences longer: ", longer)
    return np.array(all_tokens)

In [21]:
#BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_L-24_H-1024_A-16/'

In [22]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)

In [ ]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_df = pd.read_csv(os.path.join(Data_dir,"train.csv")).sample(num_to_load+valid_size,random_state=SEED)
print('loaded %d records' % len(train_df))

# Make sure all comment_text values are strings
train_df['comment_text'] = train_df['comment_text'].astype(str) 

sequences = convert_lines(train_df["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_df=train_df.fillna(0)
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
y_columns=['target', 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat', 'sexual_explicit']
difficult_cols = ['muslim', 'white', 'homosexual_gay_or_lesbian', 'black']

print("wtf1")

y_aux_train = train_df[identity_columns]

# Overall
weights = np.ones((len(train_df),)) * 0.25
# Subgroup
weights += (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) * 0.25

# Background Positive, Subgroup Negative
weights += (( (train_df['target'].values>=0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) * 0.25
# Background Negative, Subgroup Positive
weights += (( (train_df['target'].values<0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) * 0.25
loss_weight = 1.0 / weights.mean()

print("wtf2")

train_df = train_df.drop(['comment_text'],axis=1)
# convert target to 0,1
#train_df['target']=(train_df['target']>=0.5).astype(float)
print("wtf3")
#train_df['target'] = np.vstack([(train_df['target'].values>=0.5).astype(float),weights]).T
train_df['target']=(train_df['target']>=0.5).astype(float)
print("Target: ", train_df["target"])

loaded 1804874 records


In [26]:
saved_rows = "bbert_large_processed/"
np.save(saved_rows + 'sequences.npy', sequences)
np.save(saved_rows + 'weights.npy', weights)

train_df.to_hdf(saved_rows + 'train_pos.h5',key='train_data')
y_aux_train.to_hdf(saved_rows + 'y_aux_train.h5',key='train_data')

FileNotFoundError: [Errno 2] No such file or directory: 'bbert_large_processed/sequences.npy'

In [45]:
X = sequences[:num_to_load]                
y = train_df[y_columns].values[:num_to_load]
X_val = sequences[num_to_load:]                
y_val = train_df[y_columns].values[num_to_load:]

In [46]:
test_df=train_df.tail(valid_size).copy()
train_df=train_df.head(num_to_load)

In [47]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))

In [48]:
class LenMatchBatchSampler(data.BatchSampler):
    def __iter__(self):

        buckets = [[]] * 100
        yielded = 0

        for idx in self.sampler:
            count_zeros = torch.sum(self.sampler.data_source[idx][0] == 0)
            count_zeros = int(count_zeros / 64) 
            if len(buckets[count_zeros]) == 0:  buckets[count_zeros] = []

            buckets[count_zeros].append(idx)

            if len(buckets[count_zeros]) == self.batch_size:
                batch = list(buckets[count_zeros])
                yield batch
                yielded += 1
                buckets[count_zeros] = []

        batch = []
        leftover = [idx for bucket in buckets for idx in bucket]

        for idx in leftover:
            batch.append(idx)
            if len(batch) == self.batch_size:
                yielded += 1
                yield batch
                batch = []

        if len(batch) > 0 and not self.drop_last:
            yielded += 1
            yield batch

        assert len(self) == yielded, "produced an inccorect number of batches. expected %i, but yielded %i" %(len(self), yielded)

def trim_tensors(tsrs):
#     print("tsrs: ", tsrs)
#     print("tsrs[0]: ", tsrs[0])
#     print(len(tsrs[0]))
    max_len = torch.max(torch.sum( (tsrs[0] != 0  ), 1))
    if max_len > 2: 
        tsrs = [tsr[:, :max_len] for tsr in tsrs]
#     print("returned tensor: ", tsrs)
    return tsrs 

In [49]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    #print("Data shape: ", data.shape)
    #print("Target shape: ", targets.shape)
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,1:])
    return (bce_loss_1 * loss_weight) + bce_loss_2

In [50]:
output_model_file = "bert_pytorch.bin"

lr=2e-5
batch_size = 32
accumulation_steps=2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

model = BertForSequenceClassification.from_pretrained("../working",cache_dir=None,num_labels=len(y_columns))
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    #train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=None
    
    ran_sampler = data.RandomSampler(train_dataset)
    len_sampler = LenMatchBatchSampler(ran_sampler, batch_size = 32, drop_last = False)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler = len_sampler) 
    
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad() 
    for i, batch in tk0:
        #print(batch)
        tsrs = trim_tensors(batch)
        x_batch, y_batch = tuple(t.to(device) for t in tsrs)
        
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss =  F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>=0.5) == (y_batch[:,0]>=0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)


torch.save(model.state_dict(), output_model_file)


NameError: name 'amp' is not defined

In [19]:
# Run validation
# The following 2 lines are not needed but show how to download the model for prediction
model = BertForSequenceClassification(bert_config,num_labels=len(y_columns))
model.load_state_dict(torch.load(output_model_file ))
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()




BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, eleme

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, eleme

In [20]:
# From baseline kernel

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>=0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>=0.5]
    return compute_auc((subgroup_examples[label]>=0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>=0.5) & (df[label]<0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<0.5) & (df[label]>=0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>=0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>=0.5) & (df[label]>=0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<0.5) & (df[label]<0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>=0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>=0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [21]:
MODEL_NAME = 'model1'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
0,NaN,NaN,male,NaN,0
1,NaN,NaN,female,NaN,0
2,NaN,NaN,homosexual_gay_or_lesbian,NaN,0
3,NaN,NaN,christian,NaN,0
4,NaN,NaN,jewish,NaN,0
5,NaN,NaN,muslim,NaN,0
6,NaN,NaN,black,NaN,0
7,NaN,NaN,white,NaN,0
8,NaN,NaN,psychiatric_or_mental_illness,NaN,0


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [22]:
# test_df = pd.read_csv("../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv")
# test_df['comment_text'] = test_df['comment_text'].astype(str) 
# X_test = convert_lines(test_df["comment_text"].fillna("DUMMY_VALUE"), MAX_SEQUENCE_LENGTH, tokenizer)

In [23]:
# test_preds = np.zeros((len(X_test)))
# test = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long))
# test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False)
# tk0 = tqdm(test_loader)
# for i, (x_batch,) in enumerate(tk0):
#     pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
#     test_preds[i * 32:(i + 1) * 32] = pred[:, 0].detach().cpu().squeeze().numpy()

# test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [24]:
#actual test submission
# submission = pd.DataFrame.from_dict({
#     'id': test_df['id'],
#     'prediction': test_pred
# })
# submission.to_csv('submission.csv', index=False)
# submission.head()